<a href="https://colab.research.google.com/github/torchflash/MMAI831-AIOS-Assignment/blob/main/Tony/AIOS_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.Data Upload**

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
csv_url = 'https://drive.google.com/uc?export=download&id=1GPlcEcdy3e6KtS5eiH4lhPV1OjCNwT8_'
df = pd.read_csv(csv_url)
df = df.drop(df.columns[0], axis=1)

In [12]:
seed = 831

In [13]:
sales_train, sales_test = train_test_split(df, test_size=0.25, random_state=seed)

In [14]:
sales_train

,store,billboard,printout,sat,comp,price,sales
247,2101,1051,795,74,907,92,20850.424070
291,2045,1039,1131,69,918,91,20335.752820
220,2223,1274,979,67,938,95,22065.397050
840,1593,910,728,67,1022,103,11253.624270
32,1822,704,606,67,594,98,12991.913740
...,...,...,...,...,...,...,...
547,2048,810,792,68,919,108,14456.275280
737,1925,1576,926,64,471,96,23297.637780
9,1777,1111,340,72,758,108,15947.783620
659,2127,219,773,70,1004,98,8936.227514


In [15]:
sales_test

,store,billboard,printout,sat,comp,price,sales
692,2142,1032,579,69,901,103,18446.99213
805,2194,953,914,68,719,107,16403.58639
627,1905,1185,851,68,705,100,20317.30926
387,2078,1437,622,65,943,94,25472.98324
51,1854,1122,1236,64,845,98,16533.62366
...,...,...,...,...,...,...,...
716,1334,881,967,73,1024,96,12450.35271
724,2030,799,846,72,544,96,17491.66005
639,2316,867,1415,76,706,104,20622.98974
175,1790,1255,571,64,715,100,16548.68130
